# Parse CZI Files 

#### Parsing image data, metadata, and timestamps from .czi microscopy files

#### Author: Jack Crowley<br>Organization:  Zipfel & Fischbach-Teschl Research Groups, Cornell University<br>Last Updated:  5/23/2022

References:<br>
Lopez, Sergio G. https://forum.image.sc/t/reading-czi-file-in-python/39768/11<br>
AICSImageIO Contributors (2021). AICSImageIO: Image Reading, Metadata Conversion, and Image Writing for Microscopy Images in Pure Python [Computer software]. GitHub. https://github.com/AllenCellModeling/aicsimageio<br>
Gohlke, Christoph. CziFile. GitHub. https://github.com/cgohlke/czifile/tree/a70265fd430983875bf4c31955f2ad57f2592747<br>
http://www.physics.hmc.edu/~gerbode/wppriv/wp-content/uploads/2012/06/DS_ZISRAW-FileFormat_Rel_ZEN2011.pdf

In [1]:
# czi-parsing libraries
from aicsimageio.readers import CziReader
from aicsimageio.aics_image import AICSImage
import czifile as cz
#import tiffile as tf                 # TODO: write Tiff file with this library
import xml.etree.ElementTree as ET
from xml.dom import minidom

# general use
import os
import cv2
import numpy as np
from pytictoc import TicToc

# plotting libraries
import plotly
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
import kaleido

# Tk/Tcl for GUI operations
import tkinter as tk
from tkinter import filedialog
from tkinter import simpledialog

In [2]:
# Folder path information
dir_path = "I:\\2022_ALL_USER_IMAGES_HERE\\Jack Crowley\\2022_0513_lysozyme_device_lysotracker_NaI"
array_czi = [x for x in os.listdir(dir_path) if x.endswith(".czi")]

# Print a list of the .czi files in this folder
print(array_czi)

['Lysozyme_autofluorescence_1uMlysotrackerblue_flowtimeseries.czi', 'Lysozyme_autofluorescence_405.czi', 'Lysozyme_autofluorescence_405_lambda.czi', 'Lysozyme_autofluorescence_458_lambda.czi', 'Lysozyme_autofluorescence_458_lambda_2.czi', 'Lysozyme_autofluorescence_500mMKI_flowtimeseries.czi', 'Lysozyme_autofluorescence_blanktimeseries.czi', 'Lysozyme_autofluorescence_flushingout1uMlysotrackerblue_flowtimeseries.czi', 'Lysozyme_autofluorescence_flushingout500mMKI_flowtimeseries.czi', 'Lysozyme_autofluorescence_withflowtimeseries.czi']


In [3]:
# Path to individual image
filepath = dir_path + "\\" + array_czi[0]  # set index based on 
filestring = os.path.splitext(filepath)[0]
filename = os.path.basename(filestring)
print(f'File:  {filename}\n\n')

# Prepare AICSImage object from the .czi file
img = AICSImage(filepath, reader=CziReader)

# Prepare cziFile object from the .czi file
czi_obj = cz.CziFile(filepath)

# Determine image shape values of interest
NumSlices = img.shape[2]
NumChannels = img.shape[1]
PixWidth = img.physical_pixel_sizes.X
SliceHeight = img.physical_pixel_sizes.Z

# Report important data from 
print(f'Number of slices = {NumSlices}')
print(f'Number of channels = {NumChannels}')
print(f'Pixel width = {PixWidth}')
print(f'Slice height = {SliceHeight}\n')
print(f'\nPixel size is only correct if objective was set correctly in Zen')

File:  Lysozyme_autofluorescence_1uMlysotrackerblue_flowtimeseries


Number of slices = 1
Number of channels = 2
Pixel width = 5.55475093964234e-06
Slice height = 0.0


Pixel size is only correct if objective was set correctly in Zen


In [4]:
# Load metadata into xml object
img_metadata = img.metadata
xmlparse = minidom.parseString(ET.tostring(img_metadata))
prettyxml = xmlparse.toprettyxml()
print(prettyxml)

# Attachments:
print(xmlparse.getElementsByTagName("T")[0].getElementsByTagName("BinaryList")[0].childNodes[1].childNodes[0].nodeValue)

# Write image metadata to .xml file
with open("Current_Image_Metadata.xml", "w") as f:
    f.write(prettyxml)

<?xml version="1.0" ?>
<ImageDocument>
	
 
	<Metadata>
		
  
		<Version>1.0</Version>
		
  
		<Information>
			
   
			<User>
				
    
				<DisplayName>jcc453</DisplayName>
				
   
			</User>
			
   
			<Document>
				
    
				<Name>Lysozyme_autofluorescence_1uMlysotrackerblue_flowtimeseries</Name>
				
    
				<Description/>
				
    
				<Comment/>
				
    
				<UserName>jcc453</UserName>
				
    
				<CreationDate>2022-05-13T15:38:48</CreationDate>
				
    
				<SubType>Image</SubType>
				
    
				<Rating>0</Rating>
				
    
				<Title>Lysozyme_autofluorescence_1uMlysotrackerblue_flowtimeseries</Title>
				
   
			</Document>
			
   
			<Image>
				
    
				<OriginalScanData>true</OriginalScanData>
				
    
				<PixelType>Gray16</PixelType>
				
    
				<ComponentBitCount>16</ComponentBitCount>
				
    
				<SizeX>512</SizeX>
				
    
				<SizeY>512</SizeY>
				
    
				<SizeT>684</SizeT>
				
    
				<SizeC>2</SizeC>
				
    
				<MicroscopeRef Id="Microscope:0"/>

In [5]:
# Retrieve Timestamps attachment from .czi file
for attachment in czi_obj.attachments():
    if attachment.attachment_entry.name == 'TimeStamps':
        ts = attachment.data()
        ts_rel = ts - ts[0]
        print(ts_rel)

[  0.          0.4144896   0.8289792   1.2434688   1.6579584   2.072448
   2.4869376   2.9014272   3.3159168   3.7304064   4.144896    4.5593856
   4.9738752   5.3883648   5.8028544   6.217344    6.6318336   7.0463232
   7.4608128   7.8753024   8.289792    8.7042816   9.1187712   9.5332608
   9.9477504  10.36224    10.7767296  11.1912192  11.6057088  12.0201984
  12.434688   12.8491776  13.2636672  13.6781568  14.0926464  14.507136
  14.9216256  15.3361152  15.7506048  16.1650944  16.579584   16.9940736
  17.4085632  17.8230528  18.2375424  18.652032   19.0665216  19.4810112
  19.8955008  20.3099904  20.72448    21.1389696  21.5534592  21.9679488
  22.3824384  22.796928   23.2114176  23.6259072  24.0403968  24.4548864
  24.869376   25.2838656  25.6983552  26.1128448  26.5273344  26.941824
  27.3563136  27.7708032  28.1852928  28.5997824  29.014272   29.4287616
  29.8432512  30.2577408  30.6722304  31.08672    31.5012096  31.9156992
  32.3301888  32.7446784  33.159168   33.5736576  33.9

In [6]:
# Prepare image (this can take a few minutes!)
t = TicToc() #create instance of class
t.tic()
img = cz.imread(filepath)
t.toc()

img = img[0,0,:,:,:,:,:,0] # now only [C,T,Z,X,Y] remain, other axes are unnecessary
print(f'Shape:  {img.shape}')

Elapsed time is 205.097747 seconds.
Shape:  (2, 684, 1, 512, 512)
